In [1]:
import numpy as np
import pandas as pd
import csv
import quandl as q
import time
import datetime as dt
from scipy import stats
q.ApiConfig.api_key = "Ufux_HxUXZKAgFjxWhGi"

In [8]:
def score_rank(date,factor,sector): 
    data_1 = q.get_table('SHARADAR/SF1')
    data_2 = q.get_table('SHARADAR/SF2')
    data_3 = q.get_table('SHARADAR/SF3')
    data_3A = q.get_table('SHARADAR/SF3A')
    data_3B = q.get_table('SHARADAR/SF3B')
    data_SFP = q.get_table('SHARADAR/SFP')
    data_ACT = q.get_table('SHARADAR/ACTIONS')
    data_DAY = q.get_table('SHARADAR/DAILY')
    data_E = q.get_table('SHARADAR/EVENTS')
    data_indicator = q.get_table("SHARADAR/INDICATORS", paginate=True)

    data_Ticker = q.get_table('SHARADAR/TICKERS')
    sectors = list(data_Ticker['sector'].unique())

    df_SEP = pd.read_csv('/Users/guapofish/Desktop/SHARADAR_SEP.csv') #16604
    data_Ticker = data_Ticker[data_Ticker['category'] == 'Domestic']
    
    '''
    
    hc_tic = data_Ticker[data_Ticker['sector'] == 'Healthcare']
    bm_tic = data_Ticker[data_Ticker['sector'] == 'Basic Materials']
    fs_tic = data_Ticker[data_Ticker['sector'] == 'Financial Services']
    cc_tic = data_Ticker[data_Ticker['sector'] == 'Consumer Cyclical']
    ind_tic = data_Ticker[data_Ticker['sector'] == 'Industrials']
    tech_tic = data_Ticker[data_Ticker['sector'] == 'Technology']
    re_tic = data_Ticker[data_Ticker['sector'] == 'Real Estate']
    energy_tic = data_Ticker[data_Ticker['sector'] == 'Energy']
    cd_tic = data_Ticker[data_Ticker['sector'] == 'Consumer Defensive']
    cs_tic = data_Ticker[data_Ticker['sector'] == 'Communication Services']
    utility_tic = data_Ticker[data_Ticker['sector'] == 'Utilities']
    
    '''

    hc_tic = data_Ticker[data_Ticker['sector'] == sector]
    
    list1 = hc_tic.ticker.unique() #1351
    list2 = list(df_SEP.ticker.unique())
    ticker_list = [value for value in list1 if value in list2] 
    data_1 = q.get_table('SHARADAR/SF1',ticker = ticker_list, 
                         qopts = {'columns': ['ticker','calendardate','equity','sharesbas','marketcap','netinc','ncf',
                                          'cashneq','ncfdiv','ncfcommon','assets','ppnenet','inventory',
                                         'capex','netinc','ncfo','revenue','cor','sgna','epsdil']},paginate=True)

    data_1 = data_1.set_index('calendardate')
    data_1 = data_1['2013-03-31': '2018-01-01']
    data_1 = data_1.reset_index()
    data_1 = data_1.sort_values(['ticker', 'calendardate'])
    data_1 = data_1.groupby(['ticker', 'calendardate']).last().reset_index()
    data_1.set_index(['ticker','calendardate'],inplace = True)

    # book to market ratio
    data_1['B_M'] = data_1['equity']/data_1['marketcap']

    # Earning to price
    data_1['E_P'] = data_1['netinc']/data_1['marketcap']

    # cash flow to price
    data_1['CF_P']= data_1['ncf']/data_1['marketcap']

    # Total payout to price
    data_1['NO_P']= data_1['ncfdiv']/data_1['marketcap']

    # Net payout to price 
    data_1['NO_P'] = data_1['ncfdiv'] - data_1['ncfcommon']/data_1['marketcap']

    # Investment to assets
    data_1['I_A'] = (data_1['assets']/data_1['assets'].shift(1))-1

    # Net operating assets to assets
    data_1['NO_A'] = ((data_1['assets']-data_1['cashneq'])/data_1['assets'].shift(1))-1

    # Property change to assets
    data_1['PI_A'] = ((data_1['ppnenet']-data_1['ppnenet'].shift(1)+ data_1['inventory'] - data_1['inventory'].shift(1))/data_1['assets'].shift(1))-1

    # Investment growth
    data_1['IG'] = (data_1['capex']/data_1['capex'].shift(1))-1

    # Inventory growth
    data_1['IvG'] = (data_1['inventory']/data_1['inventory'].shift(1))-1

    # Inventory change to assets
    data_1['IvC'] = data_1['inventory'] - data_1['inventory'].shift(1)/((data_1['assets'] + data_1['assets'].shift(1))/2)

    # Operating accruals to assets
    data_1['O_A'] = (data_1['netinc'] - data_1['ncfo']) / data_1['assets'].shift(1)

    # Percent operating accruals
    data_1['POA'] = (data_1['netinc'] - data_1['ncfo'] )/ data_1['netinc']

    # Gross profits to assests 
    data_1['GP_A'] = (data_1['revenue'] - data_1['cor'] )/ data_1['assets']

    # Operating leverage 
    data_1['OL'] = (data_1['cor'] + data_1['sgna'])/data_1['assets']

    # SUE 
    data_1['SUE'] = data_1['epsdil']/data_1['epsdil'].shift(4)

    # ROE 
    data_1['ROE'] = data_1['netinc']/ data_1['equity'].shift(1)

    # ROA 
    data_1['ROA'] = data_1['netinc']/ data_1['assets'].shift(1)

    # NEI 
    data_1['NEI'] = data_1['netinc']/ data_1['netinc'].shift(8)

    df = data_1.reset_index()
    #df = df.set_index(['ticker','calendardate'])
    # len(df.ticker.unique()) 759

    df = df[['ticker','calendardate',str(factor)]]
    df = df[df['calendardate'] == str(date)].dropna().set_index('ticker').drop('calendardate', 1)
    df_zscore = pd.DataFrame(stats.zscore(df, axis = 0), index = df.index, columns = df.columns)
    rank = df_zscore.rank(ascending = False)
    
    df_zscore['rank'] = rank[str(factor)]
    df_zscore = df_zscore.rename(columns={str(factor):'zscore'})
    output_f = df_zscore[['zscore','rank']]
    return output_f

In [11]:
score_rank('2015-12-31','B_M','Healthcare')


,zscore,rank
ticker,,
A,-0.050529,260.0
AAC,-0.041153,232.0
ABAX,-0.064342,306.0
ABBV,-0.117302,453.0
ABC,-0.110151,438.0
ABIO,0.141713,40.0
ABMC,-0.055680,280.0
ABMD,-0.104190,426.0
ABT,-0.038151,220.0
